<a href="https://colab.research.google.com/github/JakobFrh/DS-ML_Final/blob/main/Merge_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [2]:
!pip install transformers torch
!pip install accelerate
!pip install transformers[torch] accelerate --upgrade
!python -m spacy download fr_core_news_sm
!pip install transformers datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk import word_tokenize, pos_tag
from collections import Counter
import pandas as pd
import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import torch
from torch.utils.data import Dataset
import accelerate
import transformers
from sklearn.model_selection import train_test_split
print(accelerate.__version__)
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch
from torch.utils.data import Dataset
import accelerate
import transformers
from sklearn.metrics import classification_report
from transformers import CamembertTokenizer, CamembertForSequenceClassification, AdamW

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


0.30.1


#ABC_Classification

In [4]:

train=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/train_letter.csv')

test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')




difficulty_mapping = {'A': 0, 'B': 1, 'C': 2}

train['difficulty'] = train['letter'].map(difficulty_mapping)


train_texts = train['sentence']
test_texts = test['sentence']
train_labels = train['difficulty']

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)


train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels.tolist())
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)


model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(difficulty_mapping))

optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 6
batch_size = 8

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

for epoch in range(epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
# Evaluate the model on the test set
model.eval()
test_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = tuple(t.to(device) for t in batch)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        test_preds.extend(preds.cpu().numpy())

# Map numerical predictions back to difficulty labels
inv_difficulty_mapping = {v: k for k, v in difficulty_mapping.items()}
predicted_labels = [inv_difficulty_mapping[pred] for pred in test_preds]



In [6]:
test['difficulty'] = predicted_labels
test


,id,sentence,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,C
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,B
2,2,"Et, paradoxalement, boire froid n'est pas la b...",B
3,3,"Ce n'est pas étonnant, car c'est une saison my...",A
4,4,"Le corps de Golo lui-même, d'une essence aussi...",C
...,...,...,...
1195,1195,C'est un phénomène qui trouve une accélération...,C
1196,1196,Je vais parler au serveur et voir si on peut d...,A
1197,1197,Il n'était pas comme tant de gens qui par pare...,C
1198,1198,Ils deviennent dangereux pour notre économie.,B


# Prepare the output for further prediction

In [9]:
#Result of ABC Classification
ABC=pd.read_csv('https://raw.githubusercontent.com/JakobFrh/DS-ML_Final/main/ABC_CLASIFIER.csv')
ABC['difficulty']=ABC['difficulty'].replace({ 'A1':'A', 'B1':'B','C1':'C'})
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')
test=pd.merge(ABC,test, on='id')

In [10]:
test.groupby('difficulty').size()

difficulty
A    410
B    472
C    318
dtype: int64

In [11]:
test_A1 = test[test['difficulty'] == 'A'].drop('difficulty', axis = 1)

In [12]:
test_B2 = test[test['difficulty'] == 'B'].drop('difficulty', axis = 1)

In [13]:
test_C2 = test[test['difficulty'] == 'C'].drop('difficulty', axis = 1)

# A1 Classification

In [62]:
#train_A1=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
#train_A1 = train_A1[train_A1['difficulty'].isin(['A1', 'A2'])]
#train_A1['A1'] = train_A1['difficulty'].replace({ 'A1':1, 'A2':0})
#train_A1 = train_A1.reset_index(drop=True)
#train_A1


train_A1=pd.read_csv('https://github.com/BalthazazarDuc/Data-Science-Machine-Learning/raw/main/train_A1.csv')

train_A1['A1'] = train_A1['A1'].replace({1: 'A1', 0: 'A2'})


##Pipeline

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler


preprocessor = ColumnTransformer(
    transformers=[
        ('txt', TfidfVectorizer(ngram_range=(1, 2)), 'sentence'),
        ('num', StandardScaler(), ['word_count', 'unique_words', 'comma_count'])
    ])


nlp = spacy.load('fr_core_news_sm')

def extract_features(sentence):
    doc = nlp(sentence)
    word_count = len([token.text for token in doc])
    unique_words = len(set(token.text for token in doc))
    comma_count = sentence.count(',')


    features = {
        'word_count': word_count,
        'unique_words': unique_words,
        'comma_count': comma_count,
    }

    return features

train_features_A1 = train_A1['sentence'].apply(lambda x: pd.Series(extract_features(x)))
train_A1[train_features_A1.columns] = train_features_A1


X_A1 = train_A1.drop(['A1'], axis=1)
y_A1 = train_A1['A1']

test_features_A1 = test_A1['sentence'].apply(lambda x: pd.Series(extract_features(x)))
test_A1[test_features_A1.columns] = test_features_A1

from sklearn.svm import SVC

clf_A1 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear', max_iter=10000))
])

clf_A1.fit(X_A1, y_A1)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('txt',
                                                  TfidfVectorizer(ngram_range=(1,
                                                                               2)),
                                                  'sentence'),
                                                 ('num', StandardScaler(),
                                                  ['word_count', 'unique_words',
                                                   'comma_count'])])),
                ('classifier', SVC(kernel='linear', max_iter=10000))])

In [64]:
predictions_A1 = clf_A1.predict(test_A1)

In [65]:
test_A1['difficulty'] = predictions_A1

In [66]:
test_A1.groupby('difficulty').size()

difficulty
A1    146
A2    264
dtype: int64

# Training the C2 model

In [50]:
#train_C2=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
#train_C2 = train_C2[train_C2['difficulty'].isin(['C1', 'C2'])]
#train_C2['C2'] = train_C2['difficulty'].replace({ 'C1':0, 'C2':1})
#train_C2 = train_C2.reset_index(drop=True)
#train_C2


train_C2=pd.read_csv('https://github.com/BalthazazarDuc/Data-Science-Machine-Learning/raw/main/train_C2.csv')
train_C2['C2'] = train_C2['C2'].replace({1: 'C2', 0: 'C1'})

##Pipeline

In [51]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler


preprocessor = ColumnTransformer(
    transformers=[
        ('txt', TfidfVectorizer(ngram_range=(1, 2)), 'sentence'),
        ('num', StandardScaler(), ['word_count', 'unique_words', 'comma_count'])
    ])


In [52]:
nlp = spacy.load('fr_core_news_sm')

def extract_features(sentence):
    doc = nlp(sentence)
    word_count = len([token.text for token in doc])
    unique_words = len(set(token.text for token in doc))
    comma_count = sentence.count(',')


    features = {
        'word_count': word_count,
        'unique_words': unique_words,
        'comma_count': comma_count,
    }



    return features

train_features_C2 = train_C2['sentence'].apply(lambda x: pd.Series(extract_features(x)))
train_C2[train_features_C2.columns] = train_features_C2


X_C2 = train_C2.drop(['C2'], axis=1)
y_C2 = train_C2['C2']

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [53]:
test_features_C2 = test_C2['sentence'].apply(lambda x: pd.Series(extract_features(x)))
test_C2[test_features_C2.columns] = test_features_C2

In [54]:
from sklearn.svm import SVC

clf_C2 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear', max_iter=10000))
])

In [55]:
clf_C2.fit(X_C2, y_C2)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('txt',
                                                  TfidfVectorizer(ngram_range=(1,
                                                                               2)),
                                                  'sentence'),
                                                 ('num', StandardScaler(),
                                                  ['word_count', 'unique_words',
                                                   'comma_count'])])),
                ('classifier', SVC(kernel='linear', max_iter=10000))])

In [56]:
predictions_C2 = clf_C2.predict(test_C2)

##CamemBERT C2

In [82]:
difficulty_mapping = {'C1': 0, 'C2': 1}

train_C2=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
train_C2 = train_C2[train_C2['difficulty'].isin(['C1', 'C2'])]
train_C2['C2'] = train_C2['difficulty'].replace({ 'C1':0, 'C2':1})
train_C2 = train_C2.reset_index(drop=True)
train_C2


train_texts = train_C2['sentence']
test_texts = test_C2['sentence']
train_labels = train_C2['C2']

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)


train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels.tolist())
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)


model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(difficulty_mapping))


optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 6
batch_size = 8

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

for epoch in range(epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [83]:
# Evaluate the model on the test set
model.eval()
test_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = tuple(t.to(device) for t in batch)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        test_preds.extend(preds.cpu().numpy())

inv_difficulty_mapping = {v: k for k, v in difficulty_mapping.items()}
predicted_labels = [inv_difficulty_mapping[pred] for pred in test_preds]

##Predicts

In [84]:
test_C2['difficulty_pipe']=predictions_C2
test_C2['difficulty_pipe']=test_C2['difficulty_pipe'].replace({ 0:'C1', 1:'C2'})
test_C2['difficulty_bert'] = predicted_labels
test_C2

,id,sentence,word_count,unique_words,comma_count,difficulty_pipe,difficulty_bert,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,10,9,0,C1,C2,C2
4,4,"Le corps de Golo lui-même, d'une essence aussi...",83,63,5,C2,C2,C2
5,5,"Elle jeta un cri, un petit cri, voulut se dres...",35,28,5,C1,C1,C2
13,13,"Jeune retraité de l'éducation nationale, Rog...",12,12,1,C1,C1,C1
20,20,"Et, dans le mirage confus où s'égaraient ses e...",79,55,11,C2,C2,C2
...,...,...,...,...,...,...,...,...
1174,1174,Le ressortissants d'un pays tiers à l'Union eu...,33,28,1,C1,C1,C1
1180,1180,"Mais soyons sérieux, nouvelles en bref : écono...",25,22,3,C1,C1,C1
1186,1186,"En partenariat avec l'INPES, elle a mis en pla...",32,29,3,C2,C2,C2
1191,1191,Il autorise le recours à l'IVG par l'intermédi...,28,22,3,C1,C1,C1


In [86]:
test_C2.groupby('difficulty_pipe').size()

difficulty_pipe
C1    266
C2     52
dtype: int64

In [85]:
test_C2.groupby('difficulty_bert').size()

difficulty_bert
C1    214
C2    104
dtype: int64

In [87]:
test_C2['difficulty']=test_C2[['difficulty_pipe']]

In [88]:
import numpy as np

for index, row in test_C2.iterrows():
    pipeline = row['difficulty_pipe']
    camembert = row['difficulty_bert']

    if pipeline == 'C2' and camembert == 'C2':
        test_C2.at[index, 'difficulty'] = 'C2'
    elif pipeline == 'C2' and camembert == 'C1':
        test_C2.at[index, 'difficulty'] = 'C2'
    elif pipeline == 'C1' and camembert == 'C2':
        test_C2.at[index, 'difficulty'] = 'C2'
    else:
        test_C2.at[index, 'difficulty'] = 'C1'

test_C2

,id,sentence,word_count,unique_words,comma_count,difficulty_pipe,difficulty_bert,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,10,9,0,C1,C2,C2
4,4,"Le corps de Golo lui-même, d'une essence aussi...",83,63,5,C2,C2,C2
5,5,"Elle jeta un cri, un petit cri, voulut se dres...",35,28,5,C1,C1,C1
13,13,"Jeune retraité de l'éducation nationale, Rog...",12,12,1,C1,C1,C1
20,20,"Et, dans le mirage confus où s'égaraient ses e...",79,55,11,C2,C2,C2
...,...,...,...,...,...,...,...,...
1174,1174,Le ressortissants d'un pays tiers à l'Union eu...,33,28,1,C1,C1,C1
1180,1180,"Mais soyons sérieux, nouvelles en bref : écono...",25,22,3,C1,C1,C1
1186,1186,"En partenariat avec l'INPES, elle a mis en pla...",32,29,3,C2,C2,C2
1191,1191,Il autorise le recours à l'IVG par l'intermédi...,28,22,3,C1,C1,C1


In [89]:
test_C2.groupby('difficulty').size()

difficulty
C1    203
C2    115
dtype: int64

# Sub-df's for predicting

## Big predictions frame

In [90]:
#adding B2 data only for execution
test_B2['difficulty']='B2'
result_ABC = pd.concat([test_A1, test_B2, test_C2], ignore_index=True)

### select relevant columns

In [91]:
result_ABC = result_ABC[['id', 'difficulty', 'sentence']]
result_ABC

,id,difficulty,sentence
0,6,A2,"Madame, Monsieur, Votre fils Léo arrive tous l..."
1,7,A2,Comment tu as trouvé le repas de ce midi
2,9,A1,"Je ne peux pas vous laisser dire cela, Madame."
3,10,A1,Est-ce que ça suffit?
4,11,A2,Tu as rempli le pot d'eau et tu as mis des gla...
...,...,...,...
1195,1174,C1,Le ressortissants d'un pays tiers à l'Union eu...
1196,1180,C1,"Mais soyons sérieux, nouvelles en bref : écono..."
1197,1186,C2,"En partenariat avec l'INPES, elle a mis en pla..."
1198,1191,C1,Il autorise le recours à l'IVG par l'intermédi...


### in ascending order

In [92]:
result_ABC = result_ABC.sort_values(by='id', ascending=True)

## dataframe for further prediction

In [93]:
result_A2B1B2C1 = result_ABC[result_ABC['difficulty'].isin(['A2', 'B1', 'B2', 'C1'])]


### dataframe (on this no more work needed)

In [94]:
result_A1C2 = result_ABC[~result_ABC['difficulty'].isin(['A2', 'B1', 'B2','C1'])]
result_A1C2['difficulty_pipe']=result_A1C2['difficulty']
result_A1C2

<ipython-input-94-0fe95ad8ba1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_A1C2['difficulty_pipe']=result_A1C2['difficulty']


,id,difficulty,sentence,difficulty_pipe
882,0,C2,Nous dûmes nous excuser des propos que nous eû...,C2
883,4,C2,"Le corps de Golo lui-même, d'une essence aussi...",C2
2,9,A1,"Je ne peux pas vous laisser dire cela, Madame.",A1
3,10,A1,Est-ce que ça suffit?,A1
5,14,A1,Un enfant est drôle.,A1
...,...,...,...,...
404,1182,A1,Qu'est-ce que vous pensez de cette idée ?,A1
1197,1186,C2,"En partenariat avec l'INPES, elle a mis en pla...",C2
407,1192,A1,Je n'aime pas le pain.,A1
408,1193,A1,"Ici, il fait beau et chaud.",A1


#Camembert



In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import CamembertTokenizer, CamembertForSequenceClassification, AdamW
import torch

train=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')

test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')


difficulty_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
train['difficulty'] = train['difficulty'].map(difficulty_mapping)


train_texts = train['sentence']
test_texts = test['sentence']
train_labels = train['difficulty']

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)


train_dataset =torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels.tolist())
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)


model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(difficulty_mapping))

#Training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 6
batch_size = 8


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

for epoch in range(epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
model.eval()
test_preds = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask = tuple(t.to(device) for t in batch)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        test_preds.extend(preds.cpu().numpy())

inv_difficulty_mapping = {v: k for k, v in difficulty_mapping.items()}
predicted_labels = [inv_difficulty_mapping[pred] for pred in test_preds]




In [46]:
#DataFrame with the results
results = test.copy()
results['difficulty'] = predicted_labels
print(results.groupby('difficulty').size())

difficulty
A1    270
A2    221
B1    147
B2    258
C1    187
C2    117
dtype: int64


In [47]:
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_model')

('path_to_save_model/tokenizer_config.json',
 'path_to_save_model/special_tokens_map.json',
 'path_to_save_model/sentencepiece.bpe.model',
 'path_to_save_model/added_tokens.json')

# Final df merge

In [95]:
#Results from Camembert Calssification with 6 epoch, bachsize of 8 and AdamW
Camembert=pd.read_csv('https://raw.githubusercontent.com/JakobFrh/DS-ML_Final/main/CAMEMBERT.csv')
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')
results=pd.merge(Camembert,test, on='id')

In [96]:
results['difficulty_bert']=results['difficulty']
final = pd.merge(results, result_A1C2, on='id', how='left')
print(results.groupby('difficulty').size())

difficulty
A1    172
A2    256
B1    160
B2    231
C1    238
C2    143
dtype: int64


In [97]:
import numpy as np
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')
test['difficulty']=results['difficulty']

for index, row in final.iterrows():
    pipeline = row['difficulty_pipe']
    camembert = row['difficulty_bert']

    if pipeline == 'C2' and camembert == 'C2':
        test.at[index, 'difficulty'] = 'C2'
    elif pipeline == 'C2' and camembert == 'C1':
        test.at[index, 'difficulty'] = 'C2'
    elif pipeline == 'C1' and camembert == 'C2':
        test.at[index, 'difficulty'] = 'C2'
    elif pipeline == 'C1' and camembert == 'C1':
        test.at[index, 'difficulty'] = 'C1'

    elif pipeline == 'A2' and camembert == 'A2':
        test.at[index, 'difficulty'] = 'A2'
    elif pipeline == 'A2' and camembert == 'A1':
        test.at[index, 'difficulty'] = 'A2'
    elif pipeline == 'A1' and camembert == 'A2':
        test.at[index, 'difficulty'] = 'A1'
    elif pipeline == 'A1' and camembert == 'A1':
        test.at[index, 'difficulty'] = 'A1'


test.groupby('difficulty').size()

difficulty
A1    203
A2    225
B1    160
B2    231
C1    229
C2    152
dtype: int64

In [98]:
df_predictions = test[['id', 'difficulty']]
df_predictions

,id,difficulty
0,0,C2
1,1,B1
2,2,B1
3,3,A2
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A2
1197,1197,C2
1198,1198,B2


In [101]:
df_predictions.to_csv('Final.csv', index=False)